__Logistic Regression in Keras__

In [0]:
# Import required libraries
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  train_test_split
from google.colab import drive
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import Adam,sgd
from sklearn.metrics import mean_absolute_error
from keras import layers, metrics
import keras

In [3]:
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/data/HR-Employee-Attrition.csv",sep=',')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [16]:
# Check the classes in data
data['Attrition'].unique()

array([1, 0])

In [27]:
# Conversion of Categorical values to Binary
for col in [ 'Attrition', 'BusinessTravel', 'Gender', 'JobRole', 'OverTime', 'EducationField', 'Department', 'MaritalStatus','Over18']:
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1470 non-null   int64
 1   Attrition                 1470 non-null   int8 
 2   BusinessTravel            1470 non-null   int8 
 3   DailyRate                 1470 non-null   int64
 4   Department                1470 non-null   int8 
 5   DistanceFromHome          1470 non-null   int64
 6   Education                 1470 non-null   int64
 7   EducationField            1470 non-null   int8 
 8   EmployeeCount             1470 non-null   int64
 9   EmployeeNumber            1470 non-null   int64
 10  EnvironmentSatisfaction   1470 non-null   int64
 11  Gender                    1470 non-null   int8 
 12  HourlyRate                1470 non-null   int64
 13  JobInvolvement            1470 non-null   int64
 14  JobLevel                  1470 non-null 

In [36]:
data.shape

(1470, 35)

In [0]:
# Defining the input and output layer of Keras Model
op_layer = data['Attrition'].values
ip_layer = data.drop(columns=['Attrition']).values

In [29]:
ip_layer

array([[  41,    2, 1102, ...,    4,    0,    5],
       [  49,    1,  279, ...,    7,    1,    7],
       [  37,    2, 1373, ...,    0,    0,    0],
       ...,
       [  27,    2,  155, ...,    2,    0,    3],
       [  49,    1, 1023, ...,    6,    0,    8],
       [  34,    2,  628, ...,    3,    1,    2]])

In [31]:
op_layer

array([1, 0, 1, ..., 0, 0, 0], dtype=int8)

In [0]:
# Split the dataset into Test and Train
x_train,x_test,y_train,y_test = train_test_split(ip_layer,op_layer,test_size=.3,random_state=0)

In [0]:
# Define model
def lr_algo(x_train,x_test,y_train,y_test):
    l = Sequential()
    l.add(Dense(300, activation="relu",input_shape=(34,)))
    l.add(Dropout(0.4))
    l.add(Dense(10, activation="relu"))
    l.add(Dropout(0.1))
    l.add(Dense(4, activation="relu"))
    l.add(Dense(1,activation='sigmoid'))
    l.compile(loss='binary_crossentropy', optimizer=sgd(), metrics=['accuracy'])
    l.fit(x_train, y_train, epochs=100)
    a = l.evaluate(x_test, y_test)
    return a

In [45]:
# Run the model
lr_algo(x_train,x_test,y_train,y_test)

Epoch 1/100
1029/1029 [==============================] - 0s 136us/step - loss: 469491.9923 - accuracy: 0.8095
Epoch 2/100
1029/1029 [==============================] - 0s 47us/step - loss: 0.6311 - accuracy: 0.8377
Epoch 3/100
1029/1029 [==============================] - 0s 49us/step - loss: 0.6121 - accuracy: 0.8377
Epoch 4/100
1029/1029 [==============================] - 0s 60us/step - loss: 0.5830 - accuracy: 0.8377
Epoch 5/100
1029/1029 [==============================] - 0s 53us/step - loss: 0.5722 - accuracy: 0.8377
Epoch 6/100
1029/1029 [==============================] - 0s 64us/step - loss: 0.5566 - accuracy: 0.8377
Epoch 7/100
1029/1029 [==============================] - 0s 51us/step - loss: 0.5401 - accuracy: 0.8377
Epoch 8/100
1029/1029 [==============================] - 0s 48us/step - loss: 0.5303 - accuracy: 0.8377
Epoch 9/100
1029/1029 [==============================] - 0s 50us/step - loss: 0.5244 - accuracy: 0.8377
Epoch 10/100
1029/1029 [==============================] - 

[0.43772769596030653, 0.841269850730896]